<table>
    <tr>
      <td>Minería de datos y Paradigma BigData (<b>MIN</b>) - Facultad de Informática - UCM
      </td>
      <td>
      <img src="https://biblioteca.ucm.es/data/cont/media/www/pag-88746//escudo.jpg"  width=50/>
      </td>
     </tr>
</table>




### Práctica 1 - Web scrapping de Ultimate Fighting Championship
Pablo C. Cañizares

Arturo Gómez Izquierdo y Sara Alexandra Sánchez Del Río

### Se nos encarga preparar un dataset de luchadores de la compañía para futuras decisiones estratégicas
Entre estas decisiones radican posibles `matchmaking` de combates, herramientas para ojeadores, potenciación de carreras o incluso `¡despidos!`

Empezamos por acceder a la página de estadísicas de la reputada compañias de artes marciales mixtas UFC `ufcstats.com`. Específicamente, vamos a analizar inicialmente al vigente campeón del peso pluma el hispano-georgiano `Ilia "El matador" Topuria`. Para ello, accedemos al enlace `http://ufcstats.com/fighter-details/54f64b5e283b0ce7`

In [3]:
# (Solo ejecutar si hay problemas con la version de numpy)
#import sys
#!{sys.executable} -m pip uninstall -y pyarrow numexpr bottleneck streamlit


Found existing installation: pyarrow 16.1.0
Uninstalling pyarrow-16.1.0:
  Successfully uninstalled pyarrow-16.1.0
Found existing installation: numexpr 2.8.7
Uninstalling numexpr-2.8.7:
  Successfully uninstalled numexpr-2.8.7
Found existing installation: Bottleneck 1.3.7
Uninstalling Bottleneck-1.3.7:
  Successfully uninstalled Bottleneck-1.3.7
Found existing installation: streamlit 1.37.1
Uninstalling streamlit-1.37.1:
  Successfully uninstalled streamlit-1.37.1


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


def loadPage(urlIn):
    page = requests.get(urlIn)
    soup = BeautifulSoup(page.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice
    return soup

url = "http://ufcstats.com/fighter-details/54f64b5e283b0ce7"
soup = loadPage(url) 
soup

<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
    Stats | UFC
  </title>
<meta content="" name="description"/>
<meta content="" name="viewport"/>
<link href="/blocks/main.css?ver=861173" rel="stylesheet"/>
<script src="/js/vendor/modernizr-2.6.2.min.js"></script>
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');

    ga('create', 'UA-2855164-1', 'auto');
    ga('send

In [2]:
print(soup.head.title.text)


    Stats | UFC
  


**Ejercicio 1.** (1 punto) Extrae el nombre del luchador, así como su record y alias

In [3]:
#Nombre
def getName(soup):
    nombre = soup.find(class_="b-content__title-highlight").text.strip()
    return nombre
    
#Record
def getRecord(soup):
    record = soup.find(class_="b-content__title-record").text.strip()
    return record

#NickName
def getNickName(soup):
    nickname = soup.find(class_="b-content__Nickname").text.strip()
    return nickname

nombre = getName(soup)
record = getRecord(soup)
nickname= getNickName(soup)

print(record)
print(nombre)
print(nickname)

Record: 17-0-0
Ilia Topuria
El Matador


In [4]:
#Hacemos un diccionario con las stats iniciales
def getInitialStats(soup):
    dict ={"NAME":"","RECORD":"","NICKNAME":""}
    dict["NAME"] = getName(soup)
    dict["RECORD"] = getRecord(soup)
    dict["NICKNAME"] = getNickName(soup)
    return dict

initialStatsDict = getInitialStats(soup)
print(initialStatsDict)

{'NAME': 'Ilia Topuria', 'RECORD': 'Record: 17-0-0', 'NICKNAME': 'El Matador'}


**Ejercicio 2.** (2 puntos) Extrae las caracteristicas básicas del luchador: HEIGHT, WEIGHT, REACH, STANCE, DOB

In [8]:
def cleanBasicStat(cadena):
    basicStat = ""
    if ":" in cadena:
        basicStat = cadena.split(":")[1].strip()
    return basicStat

def getBasicStatsList(soup):
    lista = soup.find(class_="b-list__box-list")
    cleanStats = []
    for elem in lista.find_all('li', recursive=False):
        cleanStats.append(cleanBasicStat(elem.text))
    return cleanStats
    


cleanStats = getBasicStatsList(soup)
print(cleanStats)

['5\' 7"', '155 lbs.', '69"', 'Orthodox', 'Jan 21, 1997']


In [9]:
#Utiliza las funciones anteriores para extraer todos los datos
def getBasicStats(soup):
    #Obtenemos la lista de estadísticas básicas
    cleanStats = getBasicStatsList(soup)
    
    #Haz un diccionario con los valores
    basicStatsDict = {"HEIGHT":"", "WEIGHT":"", "REACH":"","STANCE":"","DOB":""}

    #Asignamos los valores (despues veremos un método más pro)
    basicStatsDict["HEIGHT"] = cleanStats[0]
    basicStatsDict["WEIGHT"] = cleanStats[1]
    basicStatsDict["REACH"] = cleanStats[2]
    basicStatsDict["STANCE"] = cleanStats[3]
    basicStatsDict["DOB"] = cleanStats[4]
    return basicStatsDict
    
basicStatsDict = getBasicStats(soup)
print(basicStatsDict)

{'HEIGHT': '5\' 7"', 'WEIGHT': '155 lbs.', 'REACH': '69"', 'STANCE': 'Orthodox', 'DOB': 'Jan 21, 1997'}


**Ejercicio 3.** (2 puntos) Extrae las caracteristicas de la carrera del luchador: 
- **SLpM** - Golpes significativos conectados por minuto.
- **Str. Acc.** - Precisión en golpes significativos.
- **SApM** - Golpes significativos absorbidos por minuto.
- **Str. Def.** - Defensa ante golpes significativos (porcentaje de golpes del oponente que no conectaron).
- **TD Avg.** - Derribos promedio logrados por cada 15 minutos.
- **TD Acc.** - Precisión en derribos.
- **TD Def.** - Defensa ante derribos (porcentaje de intentos de derribo del oponente que no tuvieron éxito).
- **Sub. Avg.** - Intentos de sumisión promedio por cada 15 minutos.

In [10]:
#Obten solo los resultados, en el siguiente apartado los adaptamos con diccionarios
def getCareerStatsList(soup):
    cleanCareerStats =[]
    todo = soup.find(class_="b-list__info-box-left clearfix")
    div1 = todo.find(class_='b-list__info-box-left')
    ul1 = div1.find(class_='b-list__box-list b-list__box-list_margin-top')
    div2 = todo.find(class_='b-list__info-box-right b-list__info-box_style-margin-right')
    ul2 = div2.find(class_='b-list__box-list b-list__box-list_margin-top')
    for elem in ul1.find_all('li', recursive=False):
        aux = cleanBasicStat(elem.text)
        if aux != "":
            cleanCareerStats.append(aux)
    for elem in ul2.find_all('li', recursive=False):
        aux = cleanBasicStat(elem.text)
        if aux != "":
            cleanCareerStats.append(aux)
    return cleanCareerStats

cleanCareerStats = getCareerStatsList(soup)
print(cleanCareerStats)

['4.81', '48%', '3.83', '64%', '1.96', '61%', '93%', '1.1']


In [11]:
def getCareerStats(soup):
    cleanCareerStats = getCareerStatsList(soup)
    #Hacemos otro diccionario
    dictCS = {"SLpM":"", "Str. Acc.":"", "SApM":"","Str. Def.":"","TD Avg.":"", "TD Acc.":"", "TD Def.":"", "Sub. Avg.":""}
    dictCS = dict(zip(dictCS.keys(), cleanCareerStats))
    return dictCS
    
careerStatsDict =  getCareerStats(soup)  
print(careerStatsDict)

{'SLpM': '4.81', 'Str. Acc.': '48%', 'SApM': '3.83', 'Str. Def.': '64%', 'TD Avg.': '1.96', 'TD Acc.': '61%', 'TD Def.': '93%', 'Sub. Avg.': '1.1'}


**Ejercicio 4** (1 punto). El siguiente paso es unificar las funciones anteriores. Crea una función *extractFighterStats* que obtenga una lista de las estadísticas extraidas de la página del luchador proporcionada.

Para ello, vamos a tener que es unificar el diccionario. Existen tres metodos distintos, vamos a utilizar el más directo y más elegante

In [12]:
#Asi se mezclan los diccionarios
fighterDict = initialStatsDict | basicStatsDict | careerStatsDict
print(fighterDict)

#Y así se obtienen las claves y valores. Puede que te sirva para despues:
print(fighterDict.keys())
print(fighterDict.values())

{'NAME': 'Ilia Topuria', 'RECORD': 'Record: 17-0-0', 'NICKNAME': 'El Matador', 'HEIGHT': '5\' 7"', 'WEIGHT': '155 lbs.', 'REACH': '69"', 'STANCE': 'Orthodox', 'DOB': 'Jan 21, 1997', 'SLpM': '4.81', 'Str. Acc.': '48%', 'SApM': '3.83', 'Str. Def.': '64%', 'TD Avg.': '1.96', 'TD Acc.': '61%', 'TD Def.': '93%', 'Sub. Avg.': '1.1'}
dict_keys(['NAME', 'RECORD', 'NICKNAME', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'DOB', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def.', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.'])
dict_values(['Ilia Topuria', 'Record: 17-0-0', 'El Matador', '5\' 7"', '155 lbs.', '69"', 'Orthodox', 'Jan 21, 1997', '4.81', '48%', '3.83', '64%', '1.96', '61%', '93%', '1.1'])


In [15]:
def extractFighterStats(urlIn):
    #Obtenemos la página con soup
    page = requests.get(urlIn)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    #Adquirimos los diccionarios (llamando a las funciones ya hechas)
    dictInitial = getInitialStats(soup)
    dictBasic = getBasicStats(soup)
    dictCareer = getCareerStats(soup)

    #Hacemos un diccionario juntando los diccionarios anteriores
    dictFighter = dictInitial | dictBasic | dictCareer

    #Devolvemos una lista con los valores
    return list(dictFighter.values())

listFighter = extractFighterStats(url)
print(listFighter)

['Ilia Topuria', 'Record: 17-0-0', 'El Matador', '5\' 7"', '155 lbs.', '69"', 'Orthodox', 'Jan 21, 1997', '4.81', '48%', '3.83', '64%', '1.96', '61%', '93%', '1.1']


**Ejercicio 5** (1 puntos). Ahora vamos a probar a extraer los datos de varios luchadores. Explora la página `http://ufcstats.com/statistics/fighters`, la cual lista distintos peleadores, extrae todos los enlaces y carga las stats de los nombres. 

In [16]:
urlFighters = "http://ufcstats.com/statistics/fighters"
page = requests.get(urlFighters)
soup = BeautifulSoup(page.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice

In [18]:
def extractFighterLinksList(soup):
    tabla = soup.find(class_="b-statistics__table")
    retList = tabla.find_all(class_="b-link b-link_style_black")
    nuevaLista = []
    for elem in retList:
        nuevaLista.append(elem.get("href"))
    return nuevaLista
                
linkList = extractFighterLinksList(soup)
print(linkList)

['http://ufcstats.com/fighter-details/93fe7332d16c6ad9', 'http://ufcstats.com/fighter-details/93fe7332d16c6ad9', 'http://ufcstats.com/fighter-details/93fe7332d16c6ad9', 'http://ufcstats.com/fighter-details/15df64c02b6b0fde', 'http://ufcstats.com/fighter-details/15df64c02b6b0fde', 'http://ufcstats.com/fighter-details/15df64c02b6b0fde', 'http://ufcstats.com/fighter-details/59a9d6dac61c2540', 'http://ufcstats.com/fighter-details/59a9d6dac61c2540', 'http://ufcstats.com/fighter-details/59a9d6dac61c2540', 'http://ufcstats.com/fighter-details/4961467134abd8be', 'http://ufcstats.com/fighter-details/4961467134abd8be', 'http://ufcstats.com/fighter-details/4961467134abd8be', 'http://ufcstats.com/fighter-details/b361180739bed4b0', 'http://ufcstats.com/fighter-details/b361180739bed4b0', 'http://ufcstats.com/fighter-details/b361180739bed4b0', 'http://ufcstats.com/fighter-details/3329d692aea4dc28', 'http://ufcstats.com/fighter-details/3329d692aea4dc28', 'http://ufcstats.com/fighter-details/3329d692ae

**Ejercicio 6 (1 punto)**. Es momento de hacer nuestro mini dataset de luchadores. Para ello, itera la lista de enlaces, y extrae las stats de todos ellos. Cada vez que extraigas stats de un peleador, pon tu programa a dormir por 1 segundo ( utiliza `time.sleep(1)`)

In [19]:
import time

def extractFighterListOfLists(linkList):
    lists=[]
    i=0
    for link in linkList:
        print("link: ", link)
        stats = extractFighterStats(link)
        print("Extracted {0} stat: ".format(i),stats)
        time.sleep(1)
        i = i+1
        lists.append(stats)
    return lists

listOfLists = extractFighterListOfLists(linkList)

link:  http://ufcstats.com/fighter-details/93fe7332d16c6ad9
Extracted 0 stat:  ['Tom Aaron', 'Record: 5-3-0', '', '--', '155 lbs.', '--', '', 'Jul 13, 1978', '0.00', '0%', '0.00', '0%', '0.00', '0%', '0%', '0.0']
link:  http://ufcstats.com/fighter-details/93fe7332d16c6ad9
Extracted 1 stat:  ['Tom Aaron', 'Record: 5-3-0', '', '--', '155 lbs.', '--', '', 'Jul 13, 1978', '0.00', '0%', '0.00', '0%', '0.00', '0%', '0%', '0.0']
link:  http://ufcstats.com/fighter-details/93fe7332d16c6ad9
Extracted 2 stat:  ['Tom Aaron', 'Record: 5-3-0', '', '--', '155 lbs.', '--', '', 'Jul 13, 1978', '0.00', '0%', '0.00', '0%', '0.00', '0%', '0%', '0.0']
link:  http://ufcstats.com/fighter-details/15df64c02b6b0fde
Extracted 3 stat:  ['Danny Abbadi', 'Record: 4-6-0', 'The Assassin', '5\' 11"', '155 lbs.', '--', 'Orthodox', 'Jul 03, 1983', '3.29', '38%', '4.41', '57%', '0.00', '0%', '77%', '0.0']
link:  http://ufcstats.com/fighter-details/15df64c02b6b0fde
Extracted 4 stat:  ['Danny Abbadi', 'Record: 4-6-0', 'The

**Ejercicio 7 (1 punto)**. Ahora te toca crear un dataset, y guardarlo con el nombre `fighters.csv`

In [ ]:
import pandas as pd
from pandas import DataFrame
#Primero creamos el DF
df_fighters = DataFrame(listOfLists, columns=['NAME', 'RECORD', 'NICKNAME', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'DOB', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def.', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.'])
display(df_fighters)

              NAME          RECORD      NICKNAME  HEIGHT    WEIGHT REACH  \
0        Tom Aaron   Record: 5-3-0                    --  155 lbs.    --   
1        Tom Aaron   Record: 5-3-0                    --  155 lbs.    --   
2        Tom Aaron   Record: 5-3-0                    --  155 lbs.    --   
3     Danny Abbadi   Record: 4-6-0  The Assassin  5' 11"  155 lbs.    --   
4     Danny Abbadi   Record: 4-6-0  The Assassin  5' 11"  155 lbs.    --   
..             ...             ...           ...     ...       ...   ...   
70    Nick Agallar  Record: 24-6-0                 5' 8"  155 lbs.    --   
71    Nick Agallar  Record: 24-6-0                 5' 8"  155 lbs.    --   
72  Mariya Agapova  Record: 10-5-0  Money Mashka   5' 6"  125 lbs.   68"   
73  Mariya Agapova  Record: 10-5-0  Money Mashka   5' 6"  125 lbs.   68"   
74  Mariya Agapova  Record: 10-5-0  Money Mashka   5' 6"  125 lbs.   68"   

      STANCE           DOB  SLpM Str. Acc.  SApM Str. Def. TD Avg. TD Acc.  \
0        

In [23]:
#Ahora lo guardamos
df_fighters.to_csv('fighters.csv', sep=',', index=False)
print("Archivo CSV 'df_fighters.csv' creado.")

Archivo CSV 'df_fighters.csv' creado.


**Ejercicio 8** (1 punto). Ahora, vamos a intentar crear un dataset con los luchadores más importantes. Para ello, accede a la página `http://ufcstats.com/statistics/fighters?char=a&page=all` y selecciona únicamente los campeones. Cabe destacar que los campeones son aquellos que tienen el icono de un cinturón <img src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/belt.png" class="b-list__icon"> en la última celda

In [ ]:
# Hemos hecho desde este ejercicio en adelante, también decirte que en el ejercicio 2 debimos quitar con Ctrl+Z en algún momento la línea

# basicStat = ""        de la función cleanBasicStat que imagino que te habrá dado error por ello.

# def cleanBasicStat(cadena):
#     basicStat = ""
#     if ":" in cadena:
#         basicStat = cadena.split(":")[1].strip()
#     return basicStat

# Y la otra diferencia de cosas anteriores es haber cambiado un print(df_fighters) por un display para que se vea mejor el DataFrame.

utlTops = "http://ufcstats.com/statistics/fighters?char=a&page=all"
page = requests.get(utlTops)
soup = BeautifulSoup(page.text, 'html.parser')

def extractTopFighterLinksList(soup):
    tabla = soup.find(class_="b-statistics__table")
    filas = tabla.find_all("tr", class_="b-statistics__table-row")
    
    nuevaLista = []
    for fila in filas:
        if fila.find("img", class_="b-list__icon"):
            link = fila.find(class_="b-link b-link_style_black")
            if link:
                nuevaLista.append(link.get("href"))
                
    return nuevaLista
                
topLinkList = extractTopFighterLinksList(soup)

topListFighters = extractFighterListOfLists(topLinkList)

df_top_fighters = DataFrame(topListFighters, columns=['NAME', 'RECORD', 'NICKNAME', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'DOB', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def.', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.'])
display(df_top_fighters)


link:  http://ufcstats.com/fighter-details/399afbabc02376b5
Extracted 0 stat:  ['Tom Aspinall', 'Record: 15-3-0 (1 NC)', '', '6\' 5"', '255 lbs.', '78"', 'Orthodox', 'Apr 11, 1993', '7.63', '67%', '3.62', '56%', '2.62', '80%', '100%', '1.3']


,NAME,RECORD,NICKNAME,HEIGHT,WEIGHT,REACH,STANCE,DOB,SLpM,Str. Acc.,SApM,Str. Def.,TD Avg.,TD Acc.,TD Def.,Sub. Avg.
0,Tom Aspinall,Record: 15-3-0 (1 NC),,"6' 5""",255 lbs.,"78""",Orthodox,"Apr 11, 1993",7.63,67%,3.62,56%,2.62,80%,100%,1.3


**Ejercicio extra 1** Si ya has llegado hasta aqui, tienes que intentarlo, en el ejercicio anterior únicamente has obtenido los campeones que empiezan por la letra a. Obtiene todos los campeones de la 'a' a la 'z', modificando el caracter XX por una letra ``http://ufcstats.com/statistics/fighters?char=XX&page=all``

In [29]:
abecedario = "abcdefghijklmnopqrstuvwxyz"

champions = []
linkChampions = []

for letra in abecedario:
    url_letra = "http://ufcstats.com/statistics/fighters?char=" + letra + "&page=all"
    page = requests.get(url_letra)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    links_tops = extractTopFighterLinksList(soup)
    
    for l in links_tops:
        if l not in linkChampions:
            linkChampions.append(l)
            
champions = extractFighterListOfLists(linkChampions)

link:  http://ufcstats.com/fighter-details/399afbabc02376b5
Extracted 0 stat:  ['Tom Aspinall', 'Record: 15-3-0 (1 NC)', '', '6\' 5"', '255 lbs.', '78"', 'Orthodox', 'Apr 11, 1993', '7.63', '67%', '3.62', '56%', '2.62', '80%', '100%', '1.3']
link:  http://ufcstats.com/fighter-details/767755fd74662dbf
Extracted 1 stat:  ['Khamzat Chimaev', 'Record: 15-0-0', 'Borz', '6\' 2"', '185 lbs.', '75"', 'Orthodox', 'May 01, 1994', '4.04', '60%', '2.32', '43%', '5.29', '55%', '85%', '1.8']
link:  http://ufcstats.com/fighter-details/7447e9f28508106a
Extracted 2 stat:  ['Mackenzie Dern', 'Record: 16-5-0', '', '5\' 4"', '115 lbs.', '63"', 'Orthodox', 'Mar 24, 1993', '3.47', '41%', '3.93', '51%', '0.93', '18%', '37%', '1.1']
link:  http://ufcstats.com/fighter-details/9e8f6c728eb01124
Extracted 3 stat:  ['Justin Gaethje', 'Record: 27-5-0', 'The Highlight', '5\' 11"', '155 lbs.', '70"', 'Orthodox', 'Nov 14, 1988', '6.48', '58%', '7.05', '51%', '0.33', '40%', '74%', '0.0']
link:  http://ufcstats.com/figh

In [31]:
#Creamos el DF
df_champs = pd.DataFrame(champions, columns=['NAME', 'RECORD', 'NICKNAME', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'DOB', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def.', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.'])
display(df_champs)
#Y lo guardamos
df_champs.to_csv('fighters_champs.csv', sep=',', index=False)
print("Archivo CSV 'df_champs.csv' creado.")

,NAME,RECORD,NICKNAME,HEIGHT,WEIGHT,REACH,STANCE,DOB,SLpM,Str. Acc.,SApM,Str. Def.,TD Avg.,TD Acc.,TD Def.,Sub. Avg.
0,Tom Aspinall,Record: 15-3-0 (1 NC),,"6' 5""",255 lbs.,"78""",Orthodox,"Apr 11, 1993",7.63,67%,3.62,56%,2.62,80%,100%,1.3
1,Khamzat Chimaev,Record: 15-0-0,Borz,"6' 2""",185 lbs.,"75""",Orthodox,"May 01, 1994",4.04,60%,2.32,43%,5.29,55%,85%,1.8
2,Mackenzie Dern,Record: 16-5-0,,"5' 4""",115 lbs.,"63""",Orthodox,"Mar 24, 1993",3.47,41%,3.93,51%,0.93,18%,37%,1.1
3,Justin Gaethje,Record: 27-5-0,The Highlight,"5' 11""",155 lbs.,"70""",Orthodox,"Nov 14, 1988",6.48,58%,7.05,51%,0.33,40%,74%,0.0
4,Kayla Harrison,Record: 19-1-0,,"5' 8""",135 lbs.,"66""",Southpaw,"Jul 02, 1990",4.29,62%,1.10,66%,2.84,37%,100%,0.9
5,Islam Makhachev,Record: 28-1-0,,"5' 10""",170 lbs.,"70""",Southpaw,"Oct 27, 1991",2.45,58%,1.45,61%,3.10,56%,91%,1.0
6,Alex Pereira,Record: 13-3-0,Poatan,"6' 4""",205 lbs.,"79""",Orthodox,"Jul 07, 1987",5.16,62%,3.50,53%,0.11,50%,79%,0.2
7,Valentina Shevchenko,Record: 26-4-1,Bullet,"5' 5""",125 lbs.,"66""",Southpaw,"Mar 07, 1988",3.04,52%,2.00,63%,2.65,61%,76%,0.3
8,Ilia Topuria,Record: 17-0-0,El Matador,"5' 7""",155 lbs.,"69""",Orthodox,"Jan 21, 1997",4.81,48%,3.83,64%,1.96,61%,93%,1.1
9,Joshua Van,Record: 16-2-0,The Fearless,"5' 5""",125 lbs.,"65""",Orthodox,"Oct 10, 2001",8.84,56%,6.39,57%,0.84,63%,81%,0.0


Archivo CSV 'df_champs.csv' creado.


**Ejercicio extra 2** Extrae todos los luchadores, no solo campeones

In [33]:
abecedario = "abcdefghijklmnopqrstuvwxyz"

fighters = []
linkFighters = []

for letra in abecedario:
    url_letra = "http://ufcstats.com/statistics/fighters?char=" + letra + "&page=all"
    page = requests.get(url_letra)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    links_tops = extractFighterLinksList(soup)
    
    for l in links_tops:
        if l not in linkFighters:
            linkFighters.append(l)
            
fighters = extractFighterListOfLists(linkFighters)
    

print(fighters)


#Creamos el DF
df_fighters = pd.DataFrame(fighters, columns=['NAME', 'RECORD', 'NICKNAME', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'DOB', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def.', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.'])
display(df_fighters)
#Y lo guardamos
df_fighters.to_csv('all_fighters.csv', sep=',', index=False)
print("Archivo CSV 'all_fighters.csv' creado.")


link:  http://ufcstats.com/fighter-details/93fe7332d16c6ad9
Extracted 0 stat:  ['Tom Aaron', 'Record: 5-3-0', '', '--', '155 lbs.', '--', '', 'Jul 13, 1978', '0.00', '0%', '0.00', '0%', '0.00', '0%', '0%', '0.0']
link:  http://ufcstats.com/fighter-details/15df64c02b6b0fde
Extracted 1 stat:  ['Danny Abbadi', 'Record: 4-6-0', 'The Assassin', '5\' 11"', '155 lbs.', '--', 'Orthodox', 'Jul 03, 1983', '3.29', '38%', '4.41', '57%', '0.00', '0%', '77%', '0.0']
link:  http://ufcstats.com/fighter-details/59a9d6dac61c2540
Extracted 2 stat:  ['Nariman Abbasov', 'Record: 28-4-0', 'Bayraktar', '5\' 8"', '155 lbs.', '66"', 'Orthodox', 'Feb 01, 1994', '3.00', '20%', '5.67', '46%', '0.00', '0%', '66%', '0.0']
link:  http://ufcstats.com/fighter-details/4961467134abd8be
Extracted 3 stat:  ['Darion Abbey', 'Record: 9-5-0', '', '6\' 2"', '265 lbs.', '80"', 'Orthodox', 'Feb 25, 1993', '8.44', '50%', '14.06', '28%', '0.00', '0%', '0%', '0.0']
link:  http://ufcstats.com/fighter-details/b361180739bed4b0
Extrac

,NAME,RECORD,NICKNAME,HEIGHT,WEIGHT,REACH,STANCE,DOB,SLpM,Str. Acc.,SApM,Str. Def.,TD Avg.,TD Acc.,TD Def.,Sub. Avg.
0,Tom Aaron,Record: 5-3-0,,--,155 lbs.,--,,"Jul 13, 1978",0.00,0%,0.00,0%,0.00,0%,0%,0.0
1,Danny Abbadi,Record: 4-6-0,The Assassin,"5' 11""",155 lbs.,--,Orthodox,"Jul 03, 1983",3.29,38%,4.41,57%,0.00,0%,77%,0.0
2,Nariman Abbasov,Record: 28-4-0,Bayraktar,"5' 8""",155 lbs.,"66""",Orthodox,"Feb 01, 1994",3.00,20%,5.67,46%,0.00,0%,66%,0.0
3,Darion Abbey,Record: 9-5-0,,"6' 2""",265 lbs.,"80""",Orthodox,"Feb 25, 1993",8.44,50%,14.06,28%,0.00,0%,0%,0.0
4,David Abbott,Record: 10-15-0,Tank,"6' 0""",265 lbs.,--,Switch,"Apr 26, 1965",1.35,30%,3.55,38%,1.07,33%,66%,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,Alex Zuniga,Record: 6-3-0,,--,145 lbs.,--,,--,0.00,0%,0.00,0%,0.00,0%,0%,0.0
4448,George Zuniga,Record: 3-1-0,,"5' 9""",185 lbs.,--,,--,7.64,38%,5.45,37%,0.00,0%,100%,0.0
4449,Allan Zuniga,Record: 13-1-0,Tigre,"5' 7""",155 lbs.,"70""",Orthodox,"Apr 04, 1992",3.93,52%,1.80,61%,0.00,0%,57%,1.0
4450,Brahyam Zurcher,Record: 9-1-0,El Jarretas,"5' 10""",145 lbs.,"70""",Orthodox,"Mar 10, 1996",1.86,15%,5.57,59%,0.00,0%,0%,0.0


Archivo CSV 'all_fighters.csv' creado.


**Ejercicio extra 3** Extrae todas las peleas del evento UFC 308 con sus detalles y haz un dataframe `ufc308.csv`